In [1]:
import torch

In [2]:
import torch.nn as nn

In [3]:
#basic imports for data preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
#import data
data = pd.read_csv("../data/train.csv")
#check out the data
data.head()

,row_id,date,country,store,product,num_sold
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240


In [4]:
country = 'Germany'
public_holidays = pd.read_csv("../data/EUpublicholidays.csv")
public_holidays.head()

,Country,Holiday
0,Belgium,01/01/21
1,Belgium,04/04/21
2,Belgium,04/05/21
3,Belgium,05/01/21
4,Belgium,05/13/21


In [5]:
country_holidays = public_holidays.where(public_holidays['Country']==country)
country_holidays = country_holidays.drop(np.where(country_holidays['Country'].isnull())[0])

In [6]:
country_holidays['Holiday']=pd.to_datetime(country_holidays['Holiday'])

In [26]:
country_holidays.head(100)

,Country,Holiday
175,Germany,2017-01-01
176,Germany,2017-04-14
178,Germany,2017-05-01
179,Germany,2017-05-25
180,Germany,2017-06-05
181,Germany,2017-10-03
182,Germany,2017-10-31
183,Germany,2017-12-25
184,Germany,2017-12-26
185,Germany,2018-01-01


In [8]:
data['date']=pd.to_datetime(data['date'])
data['holiday'] = data['date'].isin(country_holidays['Holiday'])



In [9]:
#instead collect useful data from date
data['weekday'] = data['date'].map(lambda x : x.dayofweek).astype('category')
data['month'] = data['date'].map(lambda x : x.month).astype('category')
data['monthday'] = data['date'].map(lambda x : x.day).astype('category')
#year is the only continuous variable, it needs to be scaled
data['year'] = data['date'].map(lambda x : x.year - 2016).astype(float)
data['year'] = data['year'].map(lambda x : (x - 1)/(4-1))
data['product'] = data['product'].astype('category')
data['store'] = data['store'].astype('category')
dataf = data.drop('row_id',axis=1).drop('date',axis=1)
# dataf.drop('index',axis=1,inplace=True)

# dataf.drop('num_sold',axis=1,inplace=True)


dataf = dataf.where(dataf['country']==country)
dataf = dataf.drop(np.where(dataf['country'].isnull())[0])
y = dataf['num_sold'].to_numpy().astype('float32').reshape(-1,1)

In [10]:
WeekdayTransformer = OneHotEncoder(sparse=False)
MonthTransformer = OneHotEncoder(sparse=False)
MonthDayTransformer = OneHotEncoder(sparse=False)
ProductTransformer = OneHotEncoder(sparse=False)
HolidayTransformer = OneHotEncoder(sparse=False)
StoreTransformer = OneHotEncoder(sparse=False)

#create separate transformers to make it easier to drop one of the dummy variables and avoid
#multi-collinearity problems
weekday = WeekdayTransformer.fit_transform(dataf[['weekday']])[:,:-1]
month = MonthTransformer.fit_transform(dataf[['month']])[:,:-1]
monthday = MonthTransformer.fit_transform(dataf[['monthday']])[:,:-1]
product = ProductTransformer.fit_transform(dataf[['product']])[:,:-1]
holiday = HolidayTransformer.fit_transform(dataf[['holiday']])[:,:-1]
store = StoreTransformer.fit_transform(dataf[['store']])[:,:-1]


# weekday = WeekdayTransformer.fit_transform(dataf[['weekday']])
# month = MonthTransformer.fit_transform(dataf[['month']])
# monthday = MonthTransformer.fit_transform(dataf[['monthday']])
# product = ProductTransformer.fit_transform(dataf[['product']])
# holiday = HolidayTransformer.fit_transform(dataf[['holiday']])
# store = StoreTransformer.fit_transform(dataf[['store']])

# transforms = ENC.fit_transform(dataf[['weekday','month','monthday','product','store','holiday']])
year = dataf['year'].to_numpy().reshape(-1,1)

In [11]:
full_data = np.concatenate((weekday,month,monthday,product,holiday,store,year,y),axis=1)
training_data = full_data[:(len(full_data)-2000)]
test_data = full_data[(len(full_data)-2000):]

In [12]:
x_train = training_data[:,:-1][:]
y_train = training_data[:,-1:][:]
x_test = test_data[:,:-1]
y_test = test_data[:,-1:]

print(x_train.shape)

(9688, 53)


In [13]:
x_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0.])

In [14]:
from torch.utils.data import DataLoader, Dataset

In [15]:
x_train = torch.tensor(x_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)
x_test = torch.tensor(x_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)

In [16]:
y_train.shape

torch.Size([9688, 1])

In [17]:
class DataSetMaker:
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return [self.X[idx], self.y[idx]]
training_data =  DataSetMaker(x_train,y_train)
test_data = DataSetMaker(x_test,y_test)

training_data_loader = DataLoader(training_data,batch_size=32,shuffle=True)
test_data_loader = DataLoader(test_data,batch_size=32,shuffle=False)

In [18]:
neurons =512
dropout = 0.15
inputs = x_train.shape[1]

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputs, neurons),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(neurons, neurons),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(neurons, 1),
        )    
        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.15)
            if module.bias is not None:
                module.bias.data.zero_()


    def forward(self, x):
        pred = self.linear_relu_stack(x)
        return pred

model = NeuralNetwork()


In [19]:
learning_rate = 1e-3
batch_size = 8

In [20]:
loss = nn.MSELoss()

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss(pred, y).item()


    test_loss /= num_batches

    print(f"Test Error: Avg loss: {test_loss:>8f} \n")

In [23]:
model.float()
epochs = 1000
for t in range(epochs):

    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(training_data_loader, model, loss, optimizer)
    test_loop(test_data_loader, model, loss)
print("Done!")

Epoch 1
-------------------------------
loss: 88173.476562  [    0/ 9688]
loss: 18458.808594  [ 3200/ 9688]
loss: 7468.651367  [ 6400/ 9688]
loss: 9658.505859  [ 9600/ 9688]
Test Error: Avg loss: 4284.881247 

Epoch 2
-------------------------------
loss: 1259.806152  [    0/ 9688]
loss: 2447.331543  [ 3200/ 9688]
loss: 1477.382812  [ 6400/ 9688]
loss: 1474.451416  [ 9600/ 9688]
Test Error: Avg loss: 2919.575878 

Epoch 3
-------------------------------
loss: 1576.073486  [    0/ 9688]
loss: 995.328491  [ 3200/ 9688]
loss: 433.281250  [ 6400/ 9688]
loss: 502.820923  [ 9600/ 9688]
Test Error: Avg loss: 2435.664747 

Epoch 4
-------------------------------
loss: 1608.839966  [    0/ 9688]
loss: 1322.192261  [ 3200/ 9688]
loss: 1081.872070  [ 6400/ 9688]
loss: 1274.372070  [ 9600/ 9688]
Test Error: Avg loss: 2197.503305 

Epoch 5
-------------------------------
loss: 673.100891  [    0/ 9688]
loss: 1187.783325  [ 3200/ 9688]
loss: 943.460327  [ 6400/ 9688]
loss: 2011.241089  [ 9600/ 9688]

loss: 529.266235  [ 6400/ 9688]
loss: 557.787048  [ 9600/ 9688]
Test Error: Avg loss: 1216.694489 

Epoch 42
-------------------------------
loss: 708.432190  [    0/ 9688]
loss: 498.183899  [ 3200/ 9688]
loss: 155.015869  [ 6400/ 9688]
loss: 469.521942  [ 9600/ 9688]
Test Error: Avg loss: 1199.463761 

Epoch 43
-------------------------------
loss: 335.370117  [    0/ 9688]
loss: 541.058228  [ 3200/ 9688]
loss: 568.218140  [ 6400/ 9688]
loss: 625.053284  [ 9600/ 9688]
Test Error: Avg loss: 1168.858944 

Epoch 44
-------------------------------
loss: 415.632568  [    0/ 9688]
loss: 462.780731  [ 3200/ 9688]
loss: 414.273682  [ 6400/ 9688]
loss: 496.728973  [ 9600/ 9688]
Test Error: Avg loss: 1064.637345 

Epoch 45
-------------------------------
loss: 364.876434  [    0/ 9688]
loss: 846.371216  [ 3200/ 9688]
loss: 442.978638  [ 6400/ 9688]
loss: 771.781616  [ 9600/ 9688]
Test Error: Avg loss: 1117.761849 

Epoch 46
-------------------------------
loss: 390.957458  [    0/ 9688]
loss: 6

loss: 301.654724  [ 6400/ 9688]
loss: 524.643860  [ 9600/ 9688]
Test Error: Avg loss: 1046.740286 

Epoch 82
-------------------------------
loss: 731.361450  [    0/ 9688]
loss: 403.169739  [ 3200/ 9688]
loss: 360.417206  [ 6400/ 9688]
loss: 223.478668  [ 9600/ 9688]
Test Error: Avg loss: 1061.846644 

Epoch 83
-------------------------------
loss: 156.557465  [    0/ 9688]
loss: 1967.880127  [ 3200/ 9688]
loss: 436.962830  [ 6400/ 9688]
loss: 206.358795  [ 9600/ 9688]
Test Error: Avg loss: 1159.338138 

Epoch 84
-------------------------------
loss: 329.854828  [    0/ 9688]
loss: 382.411194  [ 3200/ 9688]
loss: 235.061691  [ 6400/ 9688]
loss: 303.976624  [ 9600/ 9688]
Test Error: Avg loss: 1164.302104 

Epoch 85
-------------------------------
loss: 382.023865  [    0/ 9688]
loss: 503.779083  [ 3200/ 9688]
loss: 929.491821  [ 6400/ 9688]
loss: 532.229248  [ 9600/ 9688]
Test Error: Avg loss: 1154.578453 

Epoch 86
-------------------------------
loss: 313.961060  [    0/ 9688]
loss: 

loss: 140.812866  [ 6400/ 9688]
loss: 536.319214  [ 9600/ 9688]
Test Error: Avg loss: 1159.844552 

Epoch 122
-------------------------------
loss: 114.512657  [    0/ 9688]
loss: 284.176849  [ 3200/ 9688]
loss: 85.917992  [ 6400/ 9688]
loss: 296.872253  [ 9600/ 9688]
Test Error: Avg loss: 1086.396168 

Epoch 123
-------------------------------
loss: 227.561279  [    0/ 9688]
loss: 83.719704  [ 3200/ 9688]
loss: 320.780365  [ 6400/ 9688]
loss: 235.743179  [ 9600/ 9688]
Test Error: Avg loss: 949.124986 

Epoch 124
-------------------------------
loss: 259.231018  [    0/ 9688]
loss: 236.228409  [ 3200/ 9688]
loss: 321.960388  [ 6400/ 9688]
loss: 348.631317  [ 9600/ 9688]
Test Error: Avg loss: 1017.655422 

Epoch 125
-------------------------------
loss: 371.669922  [    0/ 9688]
loss: 678.485291  [ 3200/ 9688]
loss: 592.105042  [ 6400/ 9688]
loss: 214.349518  [ 9600/ 9688]
Test Error: Avg loss: 1107.918602 

Epoch 126
-------------------------------
loss: 258.662109  [    0/ 9688]
loss:

loss: 292.670013  [ 6400/ 9688]
loss: 511.874756  [ 9600/ 9688]
Test Error: Avg loss: 957.606053 

Epoch 162
-------------------------------
loss: 232.172195  [    0/ 9688]
loss: 300.079681  [ 3200/ 9688]
loss: 311.765198  [ 6400/ 9688]
loss: 610.119873  [ 9600/ 9688]
Test Error: Avg loss: 1029.009022 

Epoch 163
-------------------------------
loss: 221.534393  [    0/ 9688]
loss: 349.098145  [ 3200/ 9688]
loss: 241.855515  [ 6400/ 9688]
loss: 296.034424  [ 9600/ 9688]
Test Error: Avg loss: 896.389490 

Epoch 164
-------------------------------
loss: 313.794586  [    0/ 9688]
loss: 678.423096  [ 3200/ 9688]
loss: 302.489166  [ 6400/ 9688]
loss: 363.731415  [ 9600/ 9688]
Test Error: Avg loss: 955.142881 

Epoch 165
-------------------------------
loss: 292.350342  [    0/ 9688]
loss: 615.446899  [ 3200/ 9688]
loss: 411.627991  [ 6400/ 9688]
loss: 561.658508  [ 9600/ 9688]
Test Error: Avg loss: 1163.251259 

Epoch 166
-------------------------------
loss: 357.508942  [    0/ 9688]
loss:

...

loss: 199.143600  [ 9600/ 9688]
Test Error: Avg loss: 850.521813 

Epoch 803
-------------------------------
loss: 110.474236  [    0/ 9688]
loss: 188.995850  [ 3200/ 9688]
loss: 251.788391  [ 6400/ 9688]
loss: 146.110535  [ 9600/ 9688]
Test Error: Avg loss: 715.517628 

Epoch 804
-------------------------------
loss: 401.848419  [    0/ 9688]
loss: 179.607529  [ 3200/ 9688]
loss: 335.441284  [ 6400/ 9688]
loss: 157.281723  [ 9600/ 9688]
Test Error: Avg loss: 804.579145 

Epoch 805
-------------------------------
loss: 233.426361  [    0/ 9688]
loss: 342.637573  [ 3200/ 9688]
loss: 130.996902  [ 6400/ 9688]
loss: 119.772095  [ 9600/ 9688]
Test Error: Avg loss: 794.411123 

Epoch 806
-------------------------------
loss: 189.527054  [    0/ 9688]
loss: 312.404968  [ 3200/ 9688]
loss: 357.912109  [ 6400/ 9688]
loss: 223.706558  [ 9600/ 9688]
Test Error: Avg loss: 854.401338 

Epoch 807
-------------------------------
loss: 184.138123  [    0/ 9688]
loss: 279.138733  [ 3200/ 9688]
loss: 9

loss: 333.777435  [ 9600/ 9688]
Test Error: Avg loss: 830.992026 

Epoch 843
-------------------------------
loss: 116.221046  [    0/ 9688]
loss: 488.849945  [ 3200/ 9688]
loss: 186.879837  [ 6400/ 9688]
loss: 148.065460  [ 9600/ 9688]
Test Error: Avg loss: 808.041510 

Epoch 844
-------------------------------
loss: 169.893982  [    0/ 9688]
loss: 265.151672  [ 3200/ 9688]
loss: 141.254669  [ 6400/ 9688]
loss: 174.881332  [ 9600/ 9688]
Test Error: Avg loss: 758.368554 

Epoch 845
-------------------------------
loss: 147.181213  [    0/ 9688]
loss: 172.235092  [ 3200/ 9688]
loss: 303.716614  [ 6400/ 9688]
loss: 205.878555  [ 9600/ 9688]
Test Error: Avg loss: 772.262350 

Epoch 846
-------------------------------
loss: 223.003296  [    0/ 9688]
loss: 220.762009  [ 3200/ 9688]
loss: 116.562431  [ 6400/ 9688]
loss: 181.265289  [ 9600/ 9688]
Test Error: Avg loss: 760.464317 

Epoch 847
-------------------------------
loss: 360.104462  [    0/ 9688]
loss: 158.364838  [ 3200/ 9688]
loss: 1

loss: 358.302643  [ 9600/ 9688]
Test Error: Avg loss: 798.623047 

Epoch 883
-------------------------------
loss: 124.200333  [    0/ 9688]
loss: 157.468536  [ 3200/ 9688]
loss: 278.965729  [ 6400/ 9688]
loss: 140.871124  [ 9600/ 9688]
Test Error: Avg loss: 791.985950 

Epoch 884
-------------------------------
loss: 139.766479  [    0/ 9688]
loss: 149.115067  [ 3200/ 9688]
loss: 233.278534  [ 6400/ 9688]
loss: 214.662521  [ 9600/ 9688]
Test Error: Avg loss: 810.851474 

Epoch 885
-------------------------------
loss: 154.130203  [    0/ 9688]
loss: 212.624130  [ 3200/ 9688]
loss: 255.274765  [ 6400/ 9688]
loss: 120.285637  [ 9600/ 9688]
Test Error: Avg loss: 820.390955 

Epoch 886
-------------------------------
loss: 212.436981  [    0/ 9688]
loss: 153.416550  [ 3200/ 9688]
loss: 172.907990  [ 6400/ 9688]
loss: 224.461792  [ 9600/ 9688]
Test Error: Avg loss: 784.148122 

Epoch 887
-------------------------------
loss: 189.716492  [    0/ 9688]
loss: 288.571594  [ 3200/ 9688]
loss: 1

loss: 198.173233  [ 6400/ 9688]
loss: 101.776909  [ 9600/ 9688]
Test Error: Avg loss: 808.464301 

Epoch 924
-------------------------------
loss: 124.575706  [    0/ 9688]
loss: 154.028091  [ 3200/ 9688]
loss: 139.666382  [ 6400/ 9688]
loss: 135.174637  [ 9600/ 9688]
Test Error: Avg loss: 785.715036 

Epoch 925
-------------------------------
loss: 263.606873  [    0/ 9688]
loss: 108.333015  [ 3200/ 9688]
loss: 149.341873  [ 6400/ 9688]
loss: 149.201050  [ 9600/ 9688]
Test Error: Avg loss: 819.025534 

Epoch 926
-------------------------------
loss: 164.720673  [    0/ 9688]
loss: 149.467300  [ 3200/ 9688]
loss: 149.282471  [ 6400/ 9688]
loss: 156.945343  [ 9600/ 9688]
Test Error: Avg loss: 819.724520 

Epoch 927
-------------------------------
loss: 355.538483  [    0/ 9688]
loss: 206.364334  [ 3200/ 9688]
loss: 133.895477  [ 6400/ 9688]
loss: 287.174011  [ 9600/ 9688]
Test Error: Avg loss: 800.420898 

Epoch 928
-------------------------------
loss: 173.314575  [    0/ 9688]
loss: 2

loss: 271.631805  [ 3200/ 9688]
loss: 299.087799  [ 6400/ 9688]
loss: 168.313171  [ 9600/ 9688]
Test Error: Avg loss: 821.091808 

Epoch 965
-------------------------------
loss: 286.307098  [    0/ 9688]
loss: 233.069061  [ 3200/ 9688]
loss: 176.896301  [ 6400/ 9688]
loss: 87.986885  [ 9600/ 9688]
Test Error: Avg loss: 843.913554 

Epoch 966
-------------------------------
loss: 451.149963  [    0/ 9688]
loss: 292.981842  [ 3200/ 9688]
loss: 250.749634  [ 6400/ 9688]
loss: 88.556328  [ 9600/ 9688]
Test Error: Avg loss: 803.927309 

Epoch 967
-------------------------------
loss: 101.852173  [    0/ 9688]
loss: 288.437439  [ 3200/ 9688]
loss: 134.528534  [ 6400/ 9688]
loss: 438.083710  [ 9600/ 9688]
Test Error: Avg loss: 818.325143 

Epoch 968
-------------------------------
loss: 221.222916  [    0/ 9688]
loss: 98.335129  [ 3200/ 9688]
loss: 104.798561  [ 6400/ 9688]
loss: 179.513550  [ 9600/ 9688]
Test Error: Avg loss: 822.495378 

Epoch 969
-------------------------------
loss: 89.4

In [24]:
PATH = f"SEPT_TPS_{country}.pt"
torch.save(model.state_dict(), PATH)

In [25]:
model(x_train[0])

tensor([641.8785], grad_fn=<AddBackward0>)